# Analysis and Visualization for Osteocyte Cell Cultures

This notebook analyzes segmented osteocyte cells from 2D cell culture videos in `data/raw/wildtype/` and `data/raw/mutant/`, computing metrics such as cell area, intensity, eccentricity, and dendritic process length for a user-specified number of frames for all videos. Outputs are saved in `data/processed/{condition}/{video_name}/` and `results/figures/{condition}/{video_name}/`. Metrics are saved in `results/metrics/{condition}/{video_name}_metrics.csv`. Optional Cellpose and random walker analysis are provided but commented out due to MPS issues on M4 chip.

In [1]:
import sys
from pathlib import Path

# Adjust the path to the root of the project
project_root = Path.cwd().parent 
sys.path.append(str(project_root))

from src.segmentation import apply_edge_filters, segment_cells, segment_cells_cellpose, refine_with_random_walker
from src.analysis import analyze_cells, analyze_dendrites
from src.visualization import plot_edge_filters, plot_combined_image, plot_contours, plot_segmentation, plot_histograms, plot_cellpose_random_walker

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from skimage.filters import sobel, scharr, prewitt, roberts, farid, laplace
from sklearn.linear_model import LinearRegression
from src.image_utils import load_video, correct_background, apply_fourier_filter, save_image

# Paths
data_dir = Path('../data/raw')
output_dir = Path('../data/processed')
results_dir = Path('../results/figures')
metrics_dir = Path('../results/metrics')
output_dir.mkdir(exist_ok=True)
results_dir.mkdir(exist_ok=True)
metrics_dir.mkdir(exist_ok=True)

# Prompt user for frame limit
response = input("Enter number of frames to process (e.g., 10, or 'all' for all frames): ").strip().lower()
max_frames = 0 if response == 'all' else int(response)

# Find videos in wildtype and mutant subfolders
video_paths = []
for condition_dir in [data_dir / 'wildtype', data_dir / 'mutant']:
    if condition_dir.exists():
        video_paths.extend(condition_dir.glob('*.mp4'))

if not video_paths:
    raise FileNotFoundError('No MP4 files found in data/raw/wildtype/ or data/raw/mutant/.')

# Crop region
crop = (40, 150, 250, 512)  # y1, y2, x1, x2

# Edge filter weights
def compute_weights(image):
    edge_filters = [sobel, scharr, prewitt, roberts, farid, laplace]
    filtered_imgs = [f(image).ravel() for f in edge_filters]
    X = np.stack(filtered_imgs, axis=1)
    y = image.ravel()
    reg = LinearRegression(fit_intercept=False)
    reg.fit(X, y)
    return reg.coef_

# Process all videos
for video_path in video_paths:
    condition = video_path.parent.name  # 'wildtype' or 'mutant'
    print(f'Processing video: {video_path.stem} ({condition})')
    # Create condition and video-specific subfolders
    condition_output_dir = output_dir / condition
    condition_results_dir = results_dir / condition
    condition_metrics_dir = metrics_dir / condition
    condition_output_dir.mkdir(exist_ok=True)
    condition_results_dir.mkdir(exist_ok=True)
    condition_metrics_dir.mkdir(exist_ok=True)
    video_output_dir = condition_output_dir / video_path.stem
    video_results_dir = condition_results_dir / video_path.stem
    video_output_dir.mkdir(exist_ok=True)
    video_results_dir.mkdir(exist_ok=True)
    # Load and preprocess video
    try:
        frames = load_video(str(video_path))
    except Exception as e:
        print(f"Error loading {video_path}: {e}. Skipping.")
        continue

    # Limit frames if specified
    frames_to_process = frames if max_frames == 0 else frames[:max_frames]

    # Process frames
    metrics = []
    for frame_idx, frame in enumerate(frames_to_process):
        print(f'Processing frame {frame_idx} of {video_path.stem} ({condition})')
        # Preprocess
        corrected = correct_background(frame)
        filtered = apply_fourier_filter(corrected)
        cropped = filtered[crop[0]:crop[1], crop[2]:crop[3]]

        # Segment full image
        combined = apply_edge_filters(filtered)
        weights = compute_weights(filtered)
        labeled, _, contours = segment_cells(combined)

        # Segment cropped region
        combined_cropped = apply_edge_filters(cropped)
        weights_cropped = compute_weights(cropped)
        labeled_cropped, _, contours_cropped = segment_cells(combined_cropped)

        # Analyze
        cell_metrics = analyze_cells(labeled, filtered)
        dendrite_metrics = analyze_dendrites(labeled > 0, index=cell_metrics.index)
        cell_metrics_cropped = analyze_cells(labeled_cropped, cropped)
        dendrite_metrics_cropped = analyze_dendrites(labeled_cropped > 0, index=cell_metrics_cropped.index)

        # Add metadata and dendritic length
        cell_metrics['video'] = video_path.stem
        cell_metrics['frame'] = frame_idx
        cell_metrics['condition'] = condition
        cell_metrics['dendritic_length'] = dendrite_metrics['dendritic_length']
        metrics.append(cell_metrics)

        cell_metrics_cropped['video'] = f'{video_path.stem}_cropped'
        cell_metrics_cropped['frame'] = frame_idx
        cell_metrics_cropped['condition'] = condition
        cell_metrics_cropped['dendritic_length'] = dendrite_metrics_cropped['dendritic_length']
        metrics.append(cell_metrics_cropped)

        # Save outputs
        frame_prefix = f'{condition}_{video_path.stem}_frame_{frame_idx:04d}'
        save_image(filtered, str(video_output_dir / f'{frame_prefix}_filtered.tif'))
        save_image(combined, str(video_output_dir / f'{frame_prefix}_combined.tif'))
        save_image(labeled, str(video_output_dir / f'{frame_prefix}_labeled.tif'))
        save_image(cropped, str(video_output_dir / f'{frame_prefix}_cropped.tif'))
        save_image(combined_cropped, str(video_output_dir / f'{frame_prefix}_combined_cropped.tif'))
        save_image(labeled_cropped, str(video_output_dir / f'{frame_prefix}_labeled_cropped.tif'))

        # Visualizations
        plot_edge_filters(filtered, video_results_dir / f'{frame_prefix}_edge_filters.png')
        plot_combined_image(filtered, combined, weights, video_results_dir / f'{frame_prefix}_combined.png')
        plot_contours(combined, contours, video_results_dir / f'{frame_prefix}_contours.png')
        plot_segmentation(filtered, combined, labeled, video_results_dir / f'{frame_prefix}_segmentation.png')
        plot_histograms(filtered, cell_metrics['area'].tolist(), video_results_dir / f'{frame_prefix}_histograms.png')

        plot_edge_filters(cropped, video_results_dir / f'{frame_prefix}_edge_filters_cropped.png')
        plot_combined_image(cropped, combined_cropped, weights_cropped, video_results_dir / f'{frame_prefix}_combined_cropped.png')
        plot_contours(combined_cropped, contours_cropped, video_results_dir / f'{frame_prefix}_contours_cropped.png')
        plot_segmentation(cropped, combined_cropped, labeled_cropped, video_results_dir / f'{frame_prefix}_segmentation_cropped.png')
        plot_histograms(cropped, cell_metrics_cropped['area'].tolist(), video_results_dir / f'{frame_prefix}_histograms_cropped.png')

    # Save metrics for this video
    if metrics:
        metrics_path = condition_metrics_dir / f'{video_path.stem}_metrics.csv'
        metrics_df = pd.concat(metrics, ignore_index=True)
        metrics_df.to_csv(metrics_path, index=False)
        print(f'Metrics saved to {metrics_path}')

# Optional: Cellpose and random walker (commented out due to MPS issues)
# try:
#     cellpose_mask = segment_cells_cellpose(cropped)
#     rw_prob = refine_with_random_walker(invert(cropped), cellpose_mask)
#     plot_cellpose_random_walker(cropped, cellpose_mask, rw_prob, video_results_dir / f'{frame_prefix}_cellpose_rw.png')
# except ImportError:
#     print('Cellpose not installed. Skipping Cellpose and random walker analysis.')
# except Exception as e:
#     print(f'Error in Cellpose/random walker analysis: {e}. Skipping.')

Processing video: Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 0 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 1 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 2 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 3 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 4 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 5 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 6 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 7 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 8 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Processing frame 9 of Confluence_Single movie_30.03.2025_no mask_C4_1 (wildtype)
Metrics saved to ../results/metrics/wildtype/Confluence_Single movie_30.03.2025_no mask_C4_1_metrics.csv
Processi